In [3]:
from kafka import KafkaProducer
import json
import csv
import time

In [4]:
# Configuraciones del productor
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    key_serializer=lambda x: x.encode('utf-8'),
    value_serializer=lambda x: json.dumps(x).encode('utf-8'),
    retries=5,
    acks='all',
)

In [6]:
def delivery_callback(marca, fila, future):
    """Callback para manejar el reconocimiento de entrega."""
    try:
        record_metadata = future  # Suponemos que future es la metadata
        print(f'###Mensaje Enviado### ===> Clave:{marca}  Mensaje:{fila}')
    except Exception as e:
        print(f'###Mensaje Erroneo### ===> Clave:{marca}  Mensaje:{fila} Error:{e}')

def produce_invoices(file_path):
    """Función para producir mensajes desde un archivo CSV."""
    with open(file_path) as archivo:
        diccionario = csv.DictReader(archivo, delimiter=',')
        for fila in diccionario:
            marca = fila["BrandName"]
            future = producer.send("test", key=marca, value=fila)
            # En este caso, 'delivery_callback()' se llamará una vez que el mensaje se haya entregado correctamente.
            # Usamos una lambda para pasar `marca`, `fila` y `future` al callback
            future.add_callback(lambda future, marca=marca, fila=fila: delivery_callback(marca, fila, future))
            time.sleep(0.5)  # Espera medio segundo entre mensajes
    # flush() se utiliza para asegurarse de que todos los mensajes en la cola del productor se envían antes de 
    # cerrar el productor. 
    producer.flush()
    print("Todos los mensajes enviados.")            

In [7]:
try:
    produce_invoices("/data/fitness_trackers.csv")  # Ruta al archivo CSV
except Exception as e:
    print(f'ERROR: {e}')
finally:
    producer.close()  # Cierra el productor al final
    print("Productor cerrado.")

###Mensaje Enviado### ===> Clave:Xiaomi  Mensaje:OrderedDict([('BrandName', 'Xiaomi'), ('DeviceType', 'FitnessBand'), ('ModelName', 'Smart Band 5'), ('Color', 'Black'), ('SellingPrice', '2,499'), ('OriginalPrice', '2,999'), ('Display', 'AMOLED Display'), ('Rating', '4.1'), ('StrapMaterial', 'Thermoplastic polyurethane'), ('AverageBatteryLifeInDays', '14'), ('Reviews', '')])
###Mensaje Enviado### ===> Clave:Xiaomi  Mensaje:OrderedDict([('BrandName', 'Xiaomi'), ('DeviceType', 'FitnessBand'), ('ModelName', 'Smart Band 4'), ('Color', 'Black'), ('SellingPrice', '2,099'), ('OriginalPrice', '2,499'), ('Display', 'AMOLED Display'), ('Rating', '4.2'), ('StrapMaterial', 'Thermoplastic polyurethane'), ('AverageBatteryLifeInDays', '14'), ('Reviews', '')])
###Mensaje Enviado### ===> Clave:Xiaomi  Mensaje:OrderedDict([('BrandName', 'Xiaomi'), ('DeviceType', 'FitnessBand'), ('ModelName', 'HMSH01GE'), ('Color', 'Black'), ('SellingPrice', '1,722'), ('OriginalPrice', '2,099'), ('Display', 'LCD Display')